In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

In [3]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [5]:
# training
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 70s 107ms/step - loss: 0.4107 - acc: 0.8156 - val_loss: 0.3381 - val_acc: 0.8516
Epoch 2/10
625/625 [==============================] - 69s 111ms/step - loss: 0.2380 - acc: 0.9093 - val_loss: 0.2678 - val_acc: 0.8900
Epoch 3/10
625/625 [==============================] - 65s 104ms/step - loss: 0.1824 - acc: 0.9343 - val_loss: 0.2786 - val_acc: 0.8942
Epoch 4/10
625/625 [==============================] - 60s 96ms/step - loss: 0.1511 - acc: 0.9463 - val_loss: 0.3029 - val_acc: 0.8852
Epoch 5/10
625/625 [==============================] - 65s 104ms/step - loss: 0.1318 - acc: 0.9552 - val_loss: 0.2931 - val_acc: 0.8860
Epoch 6/10
625/625 [==============================] - 61s 97ms/step - loss: 0.1154 - acc: 0.9614 - val_loss: 0.3103 - val_acc: 0.8928
Epoch 7/10
625/625 [==============================] - 62s 99ms/step - loss: 0.0982 - acc: 0.9678 - val_loss: 0.3862 - val_acc: 0.8860
Epoch 8/10
625/625 [==============================] - 61s 

In [9]:
keras.models.save_model(model, '2_7_sentiment_model', include_optimizer=False)

INFO:tensorflow:Assets written to: no_opti_sentiment_model\assets


INFO:tensorflow:Assets written to: no_opti_sentiment_model\assets


In [ ]:
results = model.evaluate(test_data, test_labels)
print(results)
print(test_labels)

In [ ]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

In [ ]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

In [ ]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)
